In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-16 01:02:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-16 01:02:51 (6.47 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [30]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

### Create DataFrames to match tables

In [21]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [22]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
#customers_df.show(5)

In [23]:
# Create the products_table DataFrame and drop duplicates. 
#products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df = df.dropDuplicates(['product_id'])
products_df = products_df[['product_id','product_title']]
#products_df.show(5)

In [24]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent',(to_date("review_date", 'yyyy-MM-dd').alias("review_date"))])
#review_id_df.show(5)

In [25]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
#vine_df.show(5)

In [33]:
from pyspark.sql.functions import col

bias_df = vine_df.filter(col("total_votes") >= 20)
bias_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [37]:
bias_df = bias_df.filter(col("helpful_votes") / col("total_votes") >= 0.5)
bias_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [38]:
vine_y_df = bias_df.filter(col('vine') == 'Y')
vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|     

In [40]:
vine_n_df = bias_df.filter(col('vine') == 'N')
vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|     

In [69]:
# total number of 5 star reviews where total votes > 20 and helpful_votes / total votes > 0.5
ttl_five_star = bias_df[bias_df['star_rating'] == 5].count()
ttl_all_star = bias_df.count()
print('The total number of all star reviews:{}'.format(ttl_all_star))
print('The total number of five star reviews:{}'.format(ttl_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((ttl_five_star/ttl_all_star)*100))

The total number of all star reviews:70005
The total number of five star reviews:33872
The percentage of five star reviews:48.39%


In [67]:
# % of five star reviews for Vine program (unpaid), vine == 'N'.
no_five_star = vine_n_df[vine_n_df['star_rating'] == 5].count()
no_reviews = vine_n_df.count()
print('The total number of all star reviews (not vine):{}'.format(no_reviews))
print('The total number of five star reviews (not vine):{}'.format(no_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((no_five_star/no_reviews)*100))

The total number of all star reviews (not vine):68718
The total number of five star reviews (not vine):33430
The percentage of five star reviews:48.65%


In [68]:
# % of five star reviews for Vine program (paid), vine == 'Y'.
yes_five_star = vine_y_df[vine_y_df['star_rating'] == 5].count()
yes_reviews = vine_y_df.count()
print('The total number of all star reviews (vine):{}'.format(yes_reviews))
print('The total number of five star reviews (vine):{}'.format(yes_five_star))
print('The percentage of five star reviews:{:.2f}%'.format((yes_five_star / yes_reviews) * 100))


The total number of all star reviews (vine):1287
The total number of five star reviews (vine):442
The percentage of five star reviews:34.34%


### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [26]:
# Store environmental variable
from getpass import getpass
password = getpass('A$$hATt3r')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://aws-proj.cx3jpmtuzudq.us-west-1.rds.amazonaws.com:5432/aws_reviews"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

A$$hATt3r··········


In [27]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [28]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [29]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)